### Imports necessary for running the program

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun May  5 23:01:42 2019

@author: Ahmed
"""

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

import numpy as np
import pandas as pd
'''df = pd.read_excel("MLDataSet.xlsx")
df2 = pd.read_excel("ADRs.xlsx")
df3 = pd.read_excel("DS.xlsx")
df4 = pd.read_excel("Mental.xlsx")

df5=[df, df2,  df3, df4]

result = pd.concat([df,df2,df3, df4], axis=1, join_axes=[df.index])'''
#result.to_excel('result.xlsx')


'df = pd.read_excel("MLDataSet.xlsx")\ndf2 = pd.read_excel("ADRs.xlsx")\ndf3 = pd.read_excel("DS.xlsx")\ndf4 = pd.read_excel("Mental.xlsx")\n\ndf5=[df, df2,  df3, df4]\n\nresult = pd.concat([df,df2,df3, df4], axis=1, join_axes=[df.index])'

In [18]:
def removeColumns(result):
    del result['Pain']
    del result['Content']
    del result['Filtered']
    del result['Stemmed']
    del result['big1']
    del result['big2']
    del result['small1']
    del result['small2']
    del result['Height']
    del result['Joined']
    del result['Posted']
    

### manipulating data to prepare for classification

In [21]:
def manipulate(result):
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    result['Gender']=imp_mean.fit_transform(result['Gender'].values.reshape(-1, 1))
    for i in range(len(result)):
        if result.at[i,'ADRCount']==0:
            result.at[i,'ADRCount']=0
        if result.at[i,'ADRCount']>0 and result.at[i,'ADRCount']<=3:
            result.at[i,'ADRCount']=1
        if result.at[i,'ADRCount']>3:
            result.at[i,'ADRCount']=2

        if result.at[i,'MentalCount']==0:
            result.at[i,'MentalCount']=0
        if result.at[i,'MentalCount']>0 and result.at[i,'MentalCount']<=3:
            result.at[i,'MentalCount']=1
        if result.at[i,'MentalCount']>3:
            result.at[i,'MentalCount']=2


        if result.at[i,'DieaseCount']==0:
            result.at[i,'DieaseCount']=0
        if result.at[i,'DieaseCount']>0 and result.at[i,'DieaseCount']<=3:
            result.at[i,'DieaseCount']=1
        if result.at[i,'DieaseCount']>3:
            result.at[i,'DieaseCount']=2

        if result.at[i,'Gender']=='Male':
            result.at[i,'Gender']=0
        if result.at[i,'Gender']=='Female':
            result.at[i,'Gender']=1
    result['Drug']=LabelEncoder().fit_transform(result['Drug'])
    if 'Unnamed: 0' in result:
        del result['Unnamed: 0'] 
    result['DrugFamily']=LabelEncoder().fit_transform(result['DrugFamily'])
    labels=result.iloc[:]['Pain']
    #removeColumns(result)
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
    result['Age']=imp_mean.fit_transform(result['Age'].values.reshape(-1, 1))
    result['Height']=imp_mean.fit_transform(result['Height'].values.reshape(-1, 1))
    return result

### Datasets

In [22]:
result=pd.read_excel('result_reduced.xlsx')
weighted=pd.read_excel('Weighted_reduced.xlsx')
result=manipulate(result)
weighted=manipulate(weighted)
del result['weights2']
del result['Height']


In [10]:
'''for i in weighted:
    if i not in result:
        if i != 'weights2':
            del weighted[i]
weighted.to_excel('Weighted.xlsx')'''

In [23]:
for i in weighted:
    print (weighted[i])

0      2
1      2
2      2
3      2
4      2
5      2
6      2
7      2
8      2
9      2
10     2
11     2
12     2
13     2
14     2
15     2
16     2
17     2
18     3
19     3
20     3
21     3
22     3
23     3
24     3
25     3
26     3
27     3
28     3
29     3
      ..
100    0
101    0
102    0
103    0
104    0
105    0
106    0
107    0
108    0
109    0
110    0
111    0
112    0
113    0
114    0
115    0
116    0
117    0
118    0
119    0
120    0
121    0
122    0
123    0
124    0
125    0
126    0
127    1
128    1
129    1
Name: DrugFamily, Length: 130, dtype: int32
0      0
1      0
2      0
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
14     2
15     2
16     2
17     2
18     3
19     3
20     3
21     3
22     3
23     3
24     3
25     3
26     3
27     3
28     3
29     3
      ..
100    4
101    4
102    4
103    4
104    4
105    4
106    4
107    4
108    4
109    4
110    4
111    4
112    4
113    4
11

In [2]:
def get_score(model, X_train,X_test,Y_train,Y_test):
    model.fit(X_train,Y_train)
    return model.score(X_test,Y_test)

### 10-Fold Cross validation

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
cross_val_score(RandomForestClassifier(n_estimators=400),result,labels,cv=10)

array([0.67307692, 0.8525641 , 0.82051282, 0.80128205, 0.84615385,
       0.76923077, 0.76282051, 0.90384615, 0.81935484, 0.85064935])

In [17]:
#This was used for some other no longer existing purposes
def getMissingPercentage(feature):
    #nancount = int(result[result[feature].isnull()][feature].shape[0])
    nancount = int(result[result[feature]==1][feature].shape[0])
    size=int(result.shape[0])
    print (nancount)
    print ((nancount*100)/size)
getMissingPercentage('Pvc')

163
10.468850353243417


In [5]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)

scores_rf=[]

for train_index, test_index in kf.split(result):
    X_train, X_test, Y_train, Y_test = result.iloc[train_index], result.iloc[test_index], labels.iloc[train_index],labels.iloc[test_index]
    scores_rf.append(get_score(RandomForestClassifier(n_estimators=400),X_train, X_test, Y_train, Y_test))

In [6]:
scores_rf


[0.7564102564102564,
 0.8974358974358975,
 0.8846153846153846,
 0.8205128205128205,
 0.8525641025641025,
 0.782051282051282,
 0.8269230769230769,
 0.8193548387096774,
 0.7677419354838709,
 0.8580645161290322]